# **Session #3 Homework**

### **Loading Required Libraries**

In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from sklearn.metrics import mean_squared_error, mutual_info_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression, Ridge

import warnings
warnings.filterwarnings('ignore')

### **Getting the data**

- using the New York City Airbnb open Data

In [76]:
df=pd.read_csv("AB_NYC_2019.csv")
df = df.fillna(0)
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,0,0.00,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


#### For the rest of the homework, you'll need to use only these columns:

- 'neighbourhood_group',
- 'room_type',
- 'latitude',
- 'longitude',
- 'price',
- 'minimum_nights',
- 'number_of_reviews',
- 'reviews_per_month',
- 'calculated_host_listings_count',
- 'availability_365'

Select only them and fill in the missing values with 0.

In [77]:
features = ['neighbourhood_group', 'room_type', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month', 'calculated_host_listings_count', 'availability_365']


### **Question 1**

- What is the most frequent observation (mode) for the column 'neighbourhood_group'?

In [78]:
df.neighbourhood_group.mode().astype(str)

0    Manhattan
dtype: object

### **Split the data**

- Split your data in train/val/test sets, with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value ('price') is not in your dataframe.

In [79]:
df_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)
df_train, df_val = train_test_split(df_train, test_size = 0.25, random_state = 42)

In [80]:
x_train = df_train[features].reset_index(drop = True)
x_test  = df_test[features].reset_index(drop = True)
x_val   = df_val[features].reset_index(drop = True)

In [81]:
y_train = df_train.price.values
y_test = df_test.price.values
y_val = df_val.price.values


### **Question 2**

- Create the correlation matrix for the numerical features of your train dataset.
    - In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.
- What are the two features that have the biggest correlation in this dataset?

In [82]:
cor = list(x_train.select_dtypes(include=['int', 'float']).columns)
x_train[cor].corr()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
latitude,1.000000,0.080301,0.027441,-0.006246,-0.007159,0.019375,-0.005891
longitude,0.080301,1.000000,-0.060660,0.055084,0.134642,-0.117041,0.083666
minimum_nights,0.027441,-0.060660,1.000000,-0.076020,-0.120703,0.118647,0.138901
number_of_reviews,-0.006246,0.055084,-0.076020,1.000000,0.590374,-0.073167,0.174477
reviews_per_month,-0.007159,0.134642,-0.120703,0.590374,1.000000,-0.048767,0.165376
calculated_host_listings_count,0.019375,-0.117041,0.118647,-0.073167,-0.048767,1.000000,0.225913
availability_365,-0.005891,0.083666,0.138901,0.174477,0.165376,0.225913,1.000000


In [83]:
x_train[cor].corr().abs().unstack().sort_values(ascending = False)[len(cor):len(cor)+1].index[0]

('number_of_reviews', 'reviews_per_month')

### **Make price binary**

- We need to turn the price variable from numeric into binary.
- Let's create a variable above_average which is 1 if the price is above (or equal to) 152.

In [84]:
above_average = (y_train >= 152).astype(int)

### **Question 3**

- Calculate the mutual information score with the (binarized) price for the two categorical variables that we have. Use the training set only.
- Which of these two variables has bigger score?
- Round it to 2 decimal digits using round(score, 2)

In [85]:
round(mutual_info_score(y_train, x_train.neighbourhood_group),2)

0.11

In [86]:
round(mutual_info_score(y_train, x_train.room_type),2)

0.31

### **Question 4**

- Now let's train a logistic regression
- Remember that we have two categorical variables in the data. Include them using one-hot encoding.
- Fit the model on the training dataset.
   - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
   - model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
- Calculate the accuracy on the validation dataset and rount it to 2 decimal digits.

In [87]:
above_average_train = (y_train >= 152).astype(int)
above_average_val   = (y_val >= 152).astype(int)
above_average_test  = (y_test >= 152).astype(int)

In [88]:

#on training data
train_dict = x_train.to_dict(orient = 'records')
dv = DictVectorizer(sparse=False)
dv.fit(train_dict)
x_train_ohe = dv.fit_transform(train_dict)

#fit on validation data
val_dict = x_val.to_dict(orient='records')
x_val_ohe = dv.transform(val_dict)

In [89]:
from sklearn.metrics import accuracy_score
model = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
model.fit(x_train_ohe, above_average_train)
val_preds = model.predict(x_val_ohe)
acc = accuracy_score(above_average_val,val_preds)
print(round(accuracy_score(above_average_val,val_preds),2))

0.79


### **Question 5**

- We have 9 features: 7 numerical features and 2 categorical.
- Let's find the least useful one using the feature elimination technique.
- Train a model with all these features (using the same parameters as in Q4).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.
- Which of following feature has the smallest difference?
       - neighbourhood_group
       - room_type
       - number_of_reviews
       - reviews_per_month

 - note: the difference doesn't have to be positive



In [90]:
lst = []
lst.append(['temp acc', acc])

In [91]:
for col in features:
    duplicate = x_train.copy()
    duplicate_val = x_val.copy()
    
    del duplicate[col]
    del duplicate_val[col]
    
    #init ohe on training data
    duplicate_train_dict = duplicate.to_dict(orient = 'records')
    duplicate_dv = DictVectorizer(sparse=False)
    duplicate_dv.fit(duplicate_train_dict)
    train_duplicate_ohe = dv.fit_transform(duplicate_train_dict)

    #fit on validation data
    duplicate_val_dict = duplicate_val.to_dict(orient='records')
    val_duplicate_ohe = dv.transform(duplicate_val_dict)
    
    newModel = LogisticRegression(solver='lbfgs', C=1.0, random_state=42)
    newModel.fit(train_duplicate_ohe,above_average_train)
    
    dupVal = newModel.predict(val_duplicate_ohe)
    newAcc = accuracy_score(above_average_val,dupVal)
    
    lst.append([col, newAcc])
    
print(lst)

[['temp acc', 0.7865834952449126], ['neighbourhood_group', 0.7509970344616014], ['room_type', 0.7167399529604254], ['latitude', 0.7863789753553533], ['longitude', 0.7867880151344718], ['minimum_nights', 0.7854586358523367], ['number_of_reviews', 0.7869925350240311], ['reviews_per_month', 0.7865834952449126], ['calculated_host_listings_count', 0.7866857551896922], ['availability_365', 0.7815727579507107]]


### **Question 6**

- For this question, we'll see how to use a linear regression model from Scikit-Learn
- We'll need to use the original column 'price'. Apply the logarithmic transformation to this column.
- Fit the Ridge regression model on the training data.
- This model has a parameter alpha. Let's try the following values: [0, 0.01, 0.1, 1, 10]
- Which of these alphas leads to the best RMSE on the validation set? Round your RMSE scores to 3 decimal digits.


If there are multiple options, select the smallest alpha.

In [92]:
import math
y_trainlog = np.log1p(y_train)
y_vallog = np.log1p(y_val)
y_testlog = np.log1p(y_test)

rmse_list = []

for para in [0,0.01,0.1,1,10]:
    newModel = Ridge(alpha = para)
    newModel.fit(x_train_ohe, y_trainlog)
    
    newVal = newModel.predict(x_val_ohe)
    rmse = math.sqrt(mean_squared_error(y_vallog, newVal))
    
    rmse_list.append([para, round(rmse,3)])
    
print(rmse_list) 

[[0, 0.497], [0.01, 0.497], [0.1, 0.497], [1, 0.497], [10, 0.498]]


#                                                 **DONE**